# Hyperparameters

In [1]:
hparam = {
    'epoch': 10,
    'hid_dim': 10,
    'linear_dims': [30, 16, 8],
    'report': 1,
    'lr': 0.0001,
    'train': True
}

# Configure Dataset

In [2]:
from pathlib import Path
from random import shuffle
import pandas as pd
import pickle

# Load tensor file(pickle) list
dpath = Path('./Dataset/matches_tensor')
mlist = Path('./Dataset/match_list.pkl')
if not mlist.exists():
    mat_lists = shuffle(list(dpath.glob('*')))
    with mlist.open('wb') as file: pickle.dump(mat_lists, mlist)
else:
    with mlist.open('rb') as file: mat_lists = pickle.load(file)

tr_set = mat_lists[:190000]
val_set = mat_lists[190000:200000]
te_set = mat_lists[200000:]

# Load match results as dictionaris
mat_results = pd.read_csv('./Dataset/results.csv', index_col=0).to_dict('index')

# Model

In [3]:
from model_mlp import BCEModel

M = BCEModel(hparam['linear_dims'], hparam['lr'])
param = f'mlp_{len(hparam["linear_dims"])}_layers_BCE.pkl'

# Train

In [4]:
if hparam['train']:
    for epoch in range(hparam['epoch']):
        # training process
        for idx, tr_dat in enumerate(tr_set):
            # print training progress
            print(f'[Epoch {epoch+1:3d}] ({idx+1:6d}/{len(tr_set)}) processing {tr_dat[:-4]:<30s}', end='\r')

            with open(f'{dpath}/{tr_dat}', 'rb') as feat_file:
                features = pickle.load(feat_file)
            winner = mat_results[tr_dat[:-4]]['win']

            # train
            M.train(features, winner)
        
        # validation process
        if (epoch+1)%hparam['report'] == 0: # execute validation in {hparam['report']} epoch interval
            M.reset_result()
            for idx, val_dat in enumerate(val_set):
                # print validation progress
                print(f'[Validation for epoch {epoch+1:3d}] ({idx+1:6d}/{len(val_set)}) processing {val_dat[:-4]:<30s}', end='\r') 

                with open(f'{dpath}/{val_dat}', 'rb') as feat_file:
                    features = pickle.load(feat_file)
                winner = mat_results[val_dat[:-4]]['win']
                
                # validate
                M.validate(features, winner)

            acc, pre, rec, f1, loss = M.get_result()
            print(f'[Validation for epoch {epoch+1:3d}] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')
            M.save_parameters(param)

/pytorch/aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

# Test

In [ ]:
from torch import cuda

M.load_parameters(param)
M.reset_result()
for idx, te_dat in enumerate(te_set):
    cuda.empty_cache()
    save_path = './experiment/matches_scores/mlp/'

    # print training progress
    print(f'[Testing] ({idx+1:6d}/{len(te_set)}) processing {te_dat[:-4]:<30s}', end='\r')

    with open(f'{dpath}/{te_dat}', 'rb') as feat_file:
        features = pickle.load(feat_file)
    winner = mat_results[te_dat[:-4]]['win']

    M.test(features, winner, save_path, te_dat[:-4])

acc, pre, rec, f1, loss = M.get_result()
print(f'[Test result] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')